# 🎯 Продвинутая кластеризация с оптимизацией порога расстояния

## 📋 Методология:
1. **Выбор граничного расстояния** (threshold)
2. **Разбиение выборки** на основе векторного расстояния
3. **Обучение с учителем** на полученных кластерах (классификатор)
4. **Прогноз кластеров** на тестовой выборке
5. **Оценка точности** предсказания
6. **Анализ динамики** точности от граничного расстояния

## 🔬 Алгоритм:
- Используем **иерархическую кластеризацию** или **DBSCAN** с варьируемым порогом
- Обучаем **Random Forest / SVM** на полученных метках
- Оптимизируем порог по метрикам качества классификации

## 📊 Входные данные:
CSV файл с каротажными параметрами: GR, Density, Neutron, DTP, Resistivity

## 1. Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import json
import warnings
warnings.filterwarnings('ignore')

# Кластеризация
from sklearn.cluster import AgglomerativeClustering, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, silhouette_score
)

# Классификаторы
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Расстояния
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import dendrogram, linkage

# Настройка
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("✅ Библиотеки загружены!")

## 2. Загрузка и подготовка данных

In [ ]:
# Путь к данным
input_file = 'real_well_logs.csv'

# Загрузка
try:
    df = pd.read_csv(input_file)
    print(f"✅ Данные загружены: {df.shape[0]} × {df.shape[1]}")
except FileNotFoundError:
    print("💡 Создаем ДЕМО-данные...")
    np.random.seed(42)
    n = 1000
    
    # 4 типа с разными свойствами
    cluster_types, gr_vals, rho_vals, nphi_vals, dt_vals, rt_vals = [], [], [], [], [], []
    
    for cluster_id, n_points in [(0, 250), (1, 350), (2, 200), (3, 200)]:
        cluster_types.extend([cluster_id] * n_points)
        
        if cluster_id == 0:  # Песок
            gr_vals.extend(np.random.normal(45, 8, n_points))
            rho_vals.extend(np.random.normal(2.30, 0.08, n_points))
            nphi_vals.extend(np.random.normal(0.25, 0.04, n_points))
            dt_vals.extend(np.random.normal(200, 15, n_points))
            rt_vals.extend(np.random.lognormal(1.5, 0.5, n_points))
        elif cluster_id == 1:  # Глина
            gr_vals.extend(np.random.normal(95, 12, n_points))
            rho_vals.extend(np.random.normal(2.45, 0.06, n_points))
            nphi_vals.extend(np.random.normal(0.35, 0.05, n_points))
            dt_vals.extend(np.random.normal(260, 20, n_points))
            rt_vals.extend(np.random.lognormal(0.2, 0.3, n_points))
        elif cluster_id == 2:  # Карбонат
            gr_vals.extend(np.random.normal(35, 6, n_points))
            rho_vals.extend(np.random.normal(2.70, 0.05, n_points))
            nphi_vals.extend(np.random.normal(0.12, 0.03, n_points))
            dt_vals.extend(np.random.normal(180, 12, n_points))
            rt_vals.extend(np.random.lognormal(1.2, 0.4, n_points))
        else:  # Алевролит
            gr_vals.extend(np.random.normal(65, 10, n_points))
            rho_vals.extend(np.random.normal(2.50, 0.07, n_points))
            nphi_vals.extend(np.random.normal(0.20, 0.04, n_points))
            dt_vals.extend(np.random.normal(230, 18, n_points))
            rt_vals.extend(np.random.lognormal(0.8, 0.4, n_points))
    
    indices = np.random.permutation(n)
    df = pd.DataFrame({
        'Depth': np.linspace(1000, 2000, n),
        'GR': np.array(gr_vals)[indices],
        'Density': np.array(rho_vals)[indices],
        'Neutron': np.array(nphi_vals)[indices],
        'DTP': np.array(dt_vals)[indices],
        'Resistivity': np.array(rt_vals)[indices],
        'True_Cluster': np.array(cluster_types)[indices]
    })
    print("✅ ДЕМО-данные созданы")

display(df.head())

## 3. Подготовка признаков

In [ ]:
# Выбор признаков
feature_cols = ['GR', 'Density', 'Neutron', 'DTP', 'Resistivity']
available_features = [col for col in feature_cols if col in df.columns]

print(f"📊 Признаки: {', '.join(available_features)}")

# Подготовка данных
X = df[available_features].copy()

# Логарифмирование сопротивления
if 'Resistivity' in X.columns:
    X['Resistivity'] = X['Resistivity'].clip(lower=0.01)
    X['log_Resistivity'] = np.log10(X['Resistivity'])
    X = X.drop('Resistivity', axis=1)
    print("✅ Resistivity → log10(Resistivity)")

# Удаление пропусков
X_clean = X.dropna()
print(f"✅ Данные: {len(X_clean)} точек × {X_clean.shape[1]} признаков")

# Нормализация
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)
print("✅ Данные нормализованы")

## 4. Расчет матрицы расстояний

Вычисляем евклидовы расстояния между всеми парами точек в многомерном пространстве признаков.

In [ ]:
print("🔍 Вычисление матрицы расстояний...")

# Матрица расстояний (евклидовы)
distances = pdist(X_scaled, metric='euclidean')
distance_matrix = squareform(distances)

print(f"✅ Матрица расстояний: {distance_matrix.shape}")
print(f"   Min расстояние: {distances.min():.4f}")
print(f"   Max расстояние: {distances.max():.4f}")
print(f"   Среднее расстояние: {distances.mean():.4f}")
print(f"   Медиана расстояния: {np.median(distances):.4f}")

# Визуализация распределения расстояний
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.hist(distances, bins=100, edgecolor='black', alpha=0.7)
plt.xlabel('Евклидово расстояние')
plt.ylabel('Частота')
plt.title('Распределение попарных расстояний')
plt.axvline(np.median(distances), color='r', linestyle='--', label=f'Медиана: {np.median(distances):.2f}')
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
percentiles = np.percentile(distances, [10, 25, 50, 75, 90])
plt.boxplot(distances, vert=True)
plt.ylabel('Расстояние')
plt.title('Box plot расстояний')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Перцентили расстояний:")
for p, val in zip([10, 25, 50, 75, 90], percentiles):
    print(f"   {p}%: {val:.4f}")

## 5. Функция кластеризации по граничному расстоянию

Используем **Agglomerative Clustering** с заданным порогом расстояния.

In [ ]:
def cluster_by_distance_threshold(X, distance_threshold):
    """
    Кластеризация с заданным граничным расстоянием
    
    Parameters:
    -----------
    X : array-like
        Нормализованные признаки
    distance_threshold : float
        Граничное расстояние для объединения кластеров
    
    Returns:
    --------
    labels : array
        Метки кластеров
    n_clusters : int
        Количество кластеров
    """
    clustering = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=distance_threshold,
        linkage='ward'
    )
    labels = clustering.fit_predict(X)
    n_clusters = len(np.unique(labels))
    
    return labels, n_clusters

# Тест
test_threshold = np.median(distances)
test_labels, test_n_clusters = cluster_by_distance_threshold(X_scaled, test_threshold)
print(f"✅ Функция работает! Тест с threshold={test_threshold:.2f} → {test_n_clusters} кластеров")

## 6. Перебор граничных расстояний и обучение классификаторов

**Алгоритм:**
1. Выбираем граничное расстояние
2. Разбиваем данные на кластеры
3. Делим на train/test (80/20)
4. Обучаем классификатор на train
5. Предсказываем на test
6. Оцениваем точность

In [ ]:
# Диапазон граничных расстояний для проверки
percentile_range = np.arange(5, 95, 5)  # От 5% до 90% перцентилей
distance_thresholds = [np.percentile(distances, p) for p in percentile_range]

print(f"🔍 Тестируем {len(distance_thresholds)} значений порога расстояния...")
print(f"   Диапазон: {distance_thresholds[0]:.4f} - {distance_thresholds[-1]:.4f}\n")

# Результаты
results = []

for i, threshold in enumerate(distance_thresholds):
    # 1. Кластеризация
    labels, n_clusters = cluster_by_distance_threshold(X_scaled, threshold)
    
    # Пропускаем если слишком мало или много кластеров
    if n_clusters < 2 or n_clusters > 20:
        continue
    
    # 2. Разбиение на train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, labels, test_size=0.2, random_state=42, stratify=labels
    )
    
    # 3. Обучение классификатора (Random Forest)
    clf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
    clf.fit(X_train, y_train)
    
    # 4. Прогноз
    y_pred = clf.predict(X_test)
    
    # 5. Оценка точности
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    # Cross-validation на train
    cv_scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
    cv_mean = cv_scores.mean()
    
    # Silhouette для качества кластеризации
    try:
        silhouette = silhouette_score(X_scaled, labels)
    except:
        silhouette = -1
    
    results.append({
        'threshold': threshold,
        'percentile': percentile_range[i],
        'n_clusters': n_clusters,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'cv_accuracy': cv_mean,
        'silhouette': silhouette
    })
    
    if (i + 1) % 5 == 0:
        print(f"[{i+1}/{len(distance_thresholds)}] Threshold={threshold:.3f}, Clusters={n_clusters}, Accuracy={accuracy:.3f}")

# DataFrame результатов
df_results = pd.DataFrame(results)
print(f"\n✅ Тестирование завершено! Проверено {len(df_results)} конфигураций\n")
display(df_results.head(10))

## 7. Анализ динамики точности от граничного расстояния

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. Точность vs граничное расстояние
axes[0, 0].plot(df_results['threshold'], df_results['accuracy'], 'b-o', label='Test Accuracy', linewidth=2)
axes[0, 0].plot(df_results['threshold'], df_results['cv_accuracy'], 'g--s', label='CV Accuracy', linewidth=2)
axes[0, 0].set_xlabel('Граничное расстояние')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_title('Точность vs Граничное расстояние')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Оптимум
best_idx = df_results['accuracy'].idxmax()
best_threshold = df_results.loc[best_idx, 'threshold']
best_accuracy = df_results.loc[best_idx, 'accuracy']
axes[0, 0].axvline(best_threshold, color='r', linestyle='--', alpha=0.5, label=f'Оптимум: {best_threshold:.3f}')
axes[0, 0].scatter([best_threshold], [best_accuracy], color='red', s=200, zorder=5, marker='*')

# 2. Количество кластеров vs граничное расстояние
axes[0, 1].plot(df_results['threshold'], df_results['n_clusters'], 'ro-', linewidth=2)
axes[0, 1].set_xlabel('Граничное расстояние')
axes[0, 1].set_ylabel('Количество кластеров')
axes[0, 1].set_title('Кластеры vs Граничное расстояние')
axes[0, 1].grid(alpha=0.3)
axes[0, 1].axvline(best_threshold, color='r', linestyle='--', alpha=0.5)

# 3. F1-score, Precision, Recall
axes[1, 0].plot(df_results['threshold'], df_results['f1'], 'b-o', label='F1-score', linewidth=2)
axes[1, 0].plot(df_results['threshold'], df_results['precision'], 'g-s', label='Precision', linewidth=2)
axes[1, 0].plot(df_results['threshold'], df_results['recall'], 'r-^', label='Recall', linewidth=2)
axes[1, 0].set_xlabel('Граничное расстояние')
axes[1, 0].set_ylabel('Score')
axes[1, 0].set_title('Метрики качества')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)
axes[1, 0].axvline(best_threshold, color='r', linestyle='--', alpha=0.5)

# 4. Silhouette vs Accuracy
axes[1, 1].scatter(df_results['silhouette'], df_results['accuracy'], 
                  c=df_results['n_clusters'], cmap='viridis', s=100, alpha=0.6)
axes[1, 1].set_xlabel('Silhouette Score')
axes[1, 1].set_ylabel('Accuracy')
axes[1, 1].set_title('Silhouette vs Accuracy (цвет = кол-во кластеров)')
axes[1, 1].grid(alpha=0.3)
plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1], label='N кластеров')

plt.suptitle('Анализ зависимости точности от граничного расстояния', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 8. Оптимальные параметры

In [ ]:
print("="*70)
print("🏆 ОПТИМАЛЬНЫЕ ПАРАМЕТРЫ")
print("="*70)

# Топ-5 конфигураций
top5 = df_results.nlargest(5, 'accuracy')

print("\nТоп-5 конфигураций по точности:\n")
print(top5[['threshold', 'n_clusters', 'accuracy', 'f1', 'silhouette']].to_string(index=False))

print(f"\n{'='*70}")
print(f"🎯 ОПТИМАЛЬНАЯ КОНФИГУРАЦИЯ:")
print(f"{'='*70}")

best_row = df_results.loc[best_idx]
print(f"\n  Граничное расстояние:  {best_row['threshold']:.4f}")
print(f"  Перцентиль:            {best_row['percentile']:.1f}%")
print(f"  Количество кластеров:  {best_row['n_clusters']:.0f}")
print(f"\n  📊 Метрики классификации:")
print(f"    • Accuracy:           {best_row['accuracy']:.4f}")
print(f"    • Precision:          {best_row['precision']:.4f}")
print(f"    • Recall:             {best_row['recall']:.4f}")
print(f"    • F1-score:           {best_row['f1']:.4f}")
print(f"    • CV Accuracy:        {best_row['cv_accuracy']:.4f}")
print(f"\n  📈 Качество кластеризации:")
print(f"    • Silhouette:         {best_row['silhouette']:.4f}")
print(f"\n{'='*70}")

## 9. Финальная кластеризация с оптимальным порогом

In [ ]:
print(f"🎯 Применяем оптимальное граничное расстояние: {best_threshold:.4f}\n")

# Финальная кластеризация
final_labels, final_n_clusters = cluster_by_distance_threshold(X_scaled, best_threshold)

print(f"✅ Получено {final_n_clusters} кластеров\n")

# Распределение
unique, counts = np.unique(final_labels, return_counts=True)
print("Распределение точек по кластерам:")
for cluster_id, count in zip(unique, counts):
    percentage = (count / len(final_labels)) * 100
    print(f"  Кластер {cluster_id}: {count:5d} точек ({percentage:5.2f}%)")

# Добавление к данным
X_clean['Cluster'] = final_labels

## 10. Обучение финального классификатора

In [ ]:
# Разбиение
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, final_labels, test_size=0.2, random_state=42, stratify=final_labels
)

# Обучение нескольких моделей
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='rbf', random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

print("🔬 Обучение классификаторов:\n")
model_results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    model_results[name] = {'accuracy': accuracy, 'f1': f1, 'model': model}
    print(f"  {name:20s}: Accuracy={accuracy:.4f}, F1={f1:.4f}")

# Лучшая модель
best_model_name = max(model_results, key=lambda k: model_results[k]['accuracy'])
best_model = model_results[best_model_name]['model']

print(f"\n🏆 Лучшая модель: {best_model_name}")

## 11. Confusion Matrix

In [ ]:
# Предсказания лучшей модели
y_pred_best = best_model.predict(X_test)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_best)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=[f'C{i}' for i in range(final_n_clusters)],
            yticklabels=[f'C{i}' for i in range(final_n_clusters)])
plt.title(f'Confusion Matrix ({best_model_name})', fontsize=14, fontweight='bold')
plt.xlabel('Предсказанный кластер')
plt.ylabel('Истинный кластер')
plt.tight_layout()
plt.show()

# Classification report
print("\n📊 Classification Report:\n")
print(classification_report(y_test, y_pred_best, 
                          target_names=[f'Cluster_{i}' for i in range(final_n_clusters)]))

## 12. Статистика по кластерам

In [ ]:
print("="*70)
print("📊 СТАТИСТИКА ПО КЛАСТЕРАМ")
print("="*70)

cluster_stats = {}

for cluster_id in range(final_n_clusters):
    cluster_data = X_clean[X_clean['Cluster'] == cluster_id]
    
    print(f"\n{'='*70}")
    print(f"Кластер {cluster_id} ({len(cluster_data)} точек, {len(cluster_data)/len(X_clean)*100:.1f}%)")
    print(f"{'='*70}")
    
    stats = {}
    
    for col in X_clean.columns:
        if col != 'Cluster' and col != 'log_Resistivity':
            mean_val = cluster_data[col].mean()
            std_val = cluster_data[col].std()
            min_val = cluster_data[col].min()
            max_val = cluster_data[col].max()
            
            stats[col] = {
                'mean': float(mean_val),
                'std': float(std_val),
                'min': float(min_val),
                'max': float(max_val)
            }
            
            print(f"  {col:15s}: {mean_val:8.3f} ± {std_val:6.3f}")
    
    cluster_stats[f'cluster_{cluster_id}'] = stats

print(f"\n{'='*70}")

## 13. Сохранение результатов

In [ ]:
# Конфигурация
config = {
    'metadata': {
        'method': 'distance_threshold_clustering',
        'optimal_threshold': float(best_threshold),
        'optimal_percentile': float(best_row['percentile']),
        'n_clusters': int(final_n_clusters),
        'n_points': len(X_clean),
        'best_classifier': best_model_name,
        'test_accuracy': float(best_row['accuracy']),
        'cv_accuracy': float(best_row['cv_accuracy']),
        'f1_score': float(best_row['f1']),
        'silhouette_score': float(best_row['silhouette'])
    },
    'cluster_properties': cluster_stats,
    'optimization_results': df_results.to_dict('records')
}

# Сохранение
output_file = 'optimized_clustering_config.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print(f"✅ Конфигурация сохранена: {output_file}")

# Сохранение результатов оптимизации
df_results.to_csv('optimization_results.csv', index=False)
print(f"✅ Результаты оптимизации: optimization_results.csv")

print("\n🎯 Готово!")

## 📝 Выводы

### ✅ Реализовано:
1. ✅ Выбор граничного расстояния
2. ✅ Разбиение выборки по векторному расстоянию
3. ✅ Обучение с учителем (Random Forest, SVM, GBM, KNN)
4. ✅ Прогноз кластеров на тестовой выборке
5. ✅ Оценка точности предсказания
6. ✅ Анализ динамики точности от граничного расстояния

### 🎯 Преимущества метода:
- **Оптимизация порога** по метрикам качества классификации
- **Валидация** через supervised learning
- **Объективность** - выбор количества кластеров основан на точности предсказания
- **Интерпретируемость** - понятный физический смысл граничного расстояния

### 📊 Использование:
```python
import json
with open('optimized_clustering_config.json', 'r') as f:
    config = json.load(f)

optimal_threshold = config['metadata']['optimal_threshold']
n_clusters = config['metadata']['n_clusters']
```

---
🎯 **Продвинутая кластеризация с оптимизацией** | 2025